##### Deep Learning Pipeline for Motor Imagery Classification

*Data Preparation*

EEG epochs are extracted as input data X, shaped as (trials, channels, samples), with labels y marking the task type (left hand, right hand, feet, tongue). Each trial is normalized using z-scoring to stabilize variance across trials, and an additional dimension is added so the data matches CNN input requirements (trials, channels, samples, 1). Preprocessing includes band-pass filtering (8–30 Hz), removal of EOG channels, and setting electrodes to the standard 10–20 montage.

In this extended pipeline, Common Spatial Patterns (CSP) is applied as an additional preprocessing step. Eight CSP components are extracted per subject and concatenated with the raw EEG features. This provides the model with discriminative spatial filters alongside the original time-series data.
<br>
<br>

*Model (EEGNet + CSP)*

The classifier is based on EEGNet, a compact convolutional neural network designed specifically for EEG. Temporal convolutions capture frequency-specific patterns, depthwise spatial convolutions model motor-related activity across electrodes, and separable convolutions perform efficient feature extraction. Regularization techniques such as batch normalization, dropout, and average pooling are included to stabilize training and reduce overfitting. The final dense layer with softmax activation outputs probabilities for the motor imagery classes.

By combining CSP features with EEGNet, the model leverages both domain-engineered spatial filters and deep-learned temporal–spatial features, enhancing its ability to discriminate between imagined movements.
<br>
<br>

*Training and Validation*

Instead of pooling all subjects together, models are trained and validated separately per subject. Each subject’s dataset is split into 80% training and 20% testing, with stratification to preserve class balance. Training runs for up to 100 epochs using the Adam optimizer. Evaluation includes accuracy, classification reports, and confusion matrices. Models and metadata are saved per subject for reproducibility.
<br>
<br>

*Results and Interpretation*

When training on pooled data from all subjects with EEGNet alone, the model performed close to chance (~25–35%), with cross-validation averages around 30%. Confusion matrices often showed class collapse, reflecting the difficulty of cross-subject generalization due to individual variability in EEG patterns.

With subject-specific EEGNet training (without CSP), performance improved substantially. Stronger subjects (e.g., A01T, A08T, A09T) achieved 65–75% accuracy, while weaker subjects (A02T, A05T, A06T) remained closer to chance. Still, the subject-specific approach produced higher and more stable accuracies compared to pooled training.

With the EEGNet + CSP subject-specific pipeline, accuracy increased even further:

- Stronger subjects such as A01T, A03T, A08T, and A09T achieved 75–85% accuracy, with balanced confusion matrices showing robust class separability.

- Mid-range subjects such as A02T and A06T achieved 60–65%, higher than their plain EEGNet counterparts.

- Weaker subjects (e.g., A05T) still showed difficulties but reached ~51%, an improvement over earlier models.

- In the reduced two-class scenario (A04T: left vs. right hand), accuracy rose to ~86%.

Overall, this demonstrates that CSP + EEGNet subject-specific models consistently outperform both pooled and subject-specific EEGNet alone, confirming that motor imagery classification benefits from (1) individualized training to account for subject variability and (2) integration of CSP to enhance spatial discriminability.

In [ ]:
import os
import json 
import mne
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, DepthwiseConv2D, SeparableConv2D
from tensorflow.keras.layers import BatchNormalization, Activation, AveragePooling2D, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from mne.decoding import CSP 

# Paths & Config
data_path = "BCICIV_2a_gdf/"
subjects = [f"A0{i}T.gdf" for i in range(1, 10)]  # A01T–A09T

save_dir = "saved_models"
os.makedirs(save_dir, exist_ok=True)

# Channel mapping
mapping = {
    'EEG-Fz': 'Fz', 'EEG-0': 'FC3', 'EEG-1': 'FC1', 'EEG-2': 'FCz',
    'EEG-3': 'FC2', 'EEG-4': 'FC4', 'EEG-5': 'C5', 'EEG-C3': 'C3',
    'EEG-6': 'C1', 'EEG-Cz': 'Cz', 'EEG-7': 'C2', 'EEG-C4': 'C4',
    'EEG-8': 'C6', 'EEG-9': 'CP3', 'EEG-10': 'CP1', 'EEG-11': 'CPz',
    'EEG-12': 'CP2', 'EEG-13': 'CP4', 'EEG-14': 'P1', 'EEG-Pz': 'Pz',
    'EEG-15': 'P2', 'EEG-16': 'POz'
}

# EEGNet Model
def EEGNet(nb_classes=4, Chans=22, Samples=751, dropoutRate=0.5):
    input1 = Input(shape=(Chans, Samples, 1))

    # Block 1
    block1 = Conv2D(8, (1, 64), padding='same', use_bias=False)(input1)
    block1 = BatchNormalization()(block1)
    block1 = DepthwiseConv2D((Chans, 1), use_bias=False, depth_multiplier=2)(block1)
    block1 = BatchNormalization()(block1)
    block1 = Activation('elu')(block1)
    block1 = AveragePooling2D((1, 4))(block1)
    block1 = Dropout(dropoutRate)(block1)

    # Block 2
    block2 = SeparableConv2D(16, (1, 16), padding='same', use_bias=False)(block1)
    block2 = BatchNormalization()(block2)
    block2 = Activation('elu')(block2)
    block2 = AveragePooling2D((1, 8))(block2)
    block2 = Dropout(dropoutRate)(block2)

    # Classification
    flatten = Flatten()(block2)
    dense = Dense(nb_classes, activation='softmax')(flatten)

    return Model(inputs=input1, outputs=dense)

# Subject-specific Training
for subj in subjects:
    subj_name = subj.replace(".gdf", "")
    print(f"\n=== Training on {subj_name} ===")
    raw = mne.io.read_raw_gdf(os.path.join(data_path, subj), preload=True)

    # Preprocessing
    raw.filter(8., 30., method="fir")
    raw.rename_channels(mapping)
    for ch in ['EOG-left', 'EOG-central', 'EOG-right']:
        if ch in raw.ch_names:
            raw.drop_channels([ch])
    raw.set_montage('standard_1020')

    # Extract events
    events, event_id_raw = mne.events_from_annotations(raw)
    print("Event IDs found:", event_id_raw)

    all_event_id = {'left_hand': 7, 'right_hand': 8, 'feet': 9, 'tongue': 10}
    available_ids = set(np.unique(events[:, -1]))
    event_id = {k: v for k, v in all_event_id.items() if v in available_ids}
    print("Using event_id for this subject:", event_id)

    if len(event_id) < 2:
        print("⚠️ Skipping subject because too few classes available.")
        continue

    # Epoching
    picks = mne.pick_types(raw.info, eeg=True, eog=False)
    epochs = mne.Epochs(raw, events, event_id=event_id,
                        tmin=0.5, tmax=3.5,
                        picks=picks, baseline=None, preload=True)

    X = epochs.get_data()  # (trials, chans, samples)

    # Remap labels
    label_map = {v: i for i, v in enumerate(event_id.values())}
    y = np.array([label_map[e[-1]] for e in epochs.events], dtype=int)

    # CSP step
    n_csp = 8
    csp = CSP(n_components=n_csp, reg=None, log=False, norm_trace=False)
    X_csp = csp.fit_transform(X, y)   # (trials, n_csp)

    # Expand CSP features along time (751 samples)
    X_csp = np.repeat(X_csp[:, :, np.newaxis], X.shape[2], axis=2)  # (trials, n_csp, samples)

    # Normalize raw EEG
    X = (X - X.mean(axis=2, keepdims=True)) / (X.std(axis=2, keepdims=True) + 1e-6)

    # Concatenate EEG + CSP
    X_concat = np.concatenate([X, X_csp], axis=1)  # (trials, chans+n_csp, samples)

    # Add channel dimension
    X_concat = X_concat[..., np.newaxis]

    print(f"  -> Final X shape: {X_concat.shape}, y shape: {y.shape}, classes: {np.unique(y)}")

    # Split into train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X_concat, y, test_size=0.2, random_state=42, stratify=y
    )

    # Build model
    model = EEGNet(nb_classes=len(np.unique(y)),
                   Chans=X_concat.shape[1],
                   Samples=X_concat.shape[2])
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(0.001),
                  metrics=['accuracy'])

    # Train
    model.fit(
        X_train, y_train,
        batch_size=32,
        epochs=100,
        validation_data=(X_test, y_test),
        verbose=1
    )

    # Evaluate
    y_pred = np.argmax(model.predict(X_test), axis=1)
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    #  Save model & metadata 
    model_path = os.path.join(save_dir, f"EEGNet_CSP_{subj_name}.h5")
    model.save(model_path)

    meta = {
        "subject": subj_name,
        "event_id": event_id,
        "label_map": {str(k): int(v) for k, v in label_map.items()},
        "classes": list(event_id.keys()),
        "ch_names": epochs.ch_names,
        "input_shape": [int(X_concat.shape[1]), int(X_concat.shape[2]), 1],
        "n_csp": n_csp
    }
    with open(os.path.join(save_dir, f"EEGNet_CSP_{subj_name}_meta.json"), "w") as f:
        json.dump(meta, f, indent=2)

    print(f"✅ Saved model to {model_path}")
    print(f"✅ Saved metadata to {os.path.join(save_dir, f'EEGNet_CSP_{subj_name}_meta.json')}")



=== Training on A01T ===
Extracting EDF parameters from c:\Users\hamih\OneDrive\Desktop\personal projects\BCI\BCICIV_2a_gdf\A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


c:\Users\hamih\anaconda3\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)

Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
Event IDs found: {'1023': 1, '1072': 2, '276': 3, '277': 4, '32766': 5, '768': 6, '769': 7, '770': 8, '771': 9, '772': 10}
Using event_id for this subject: {'left_hand': 7, 'right_hand': 8, 'feet': 9, 'tongue': 10}
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Using data f


Classification Report:
              precision    recall  f1-score   support

           0     0.8750    0.4667    0.6087        15
           1     0.5652    0.9286    0.7027        14
           2     0.6667    0.6667    0.6667        15
           3     0.8333    0.7143    0.7692        14

    accuracy                         0.6897        58
   macro avg     0.7351    0.6940    0.6868        58
weighted avg     0.7363    0.6897    0.6851        58

Confusion Matrix:
[[ 7  7  1  0]
 [ 1 13  0  0]
 [ 0  3 10  2]
 [ 0  0  4 10]]
✅ Saved model to saved_models\EEGNet_CSP_A01T.h5
✅ Saved metadata to saved_models\EEGNet_CSP_A01T_meta.json

=== Training on A02T ===
Extracting EDF parameters from c:\Users\hamih\OneDrive\Desktop\personal projects\BCI\BCICIV_2a_gdf\A02T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG

c:\Users\hamih\anaconda3\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


...

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)

Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
Event IDs found: {'1023': 1, '1072': 2, '276': 3, '277': 4, '32766': 5, '768': 6, '769': 7, '770': 8, '771': 9, '772': 10}
Using event_id for this subject: {'left_hand': 7, 'right_hand': 8, 'feet': 9, 'tongue': 10}
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Using data f


Classification Report:
              precision    recall  f1-score   support

           0     0.9091    0.6667    0.7692        15
           1     0.8667    0.9286    0.8966        14
           2     0.8000    0.8000    0.8000        15
           3     0.8235    1.0000    0.9032        14

    accuracy                         0.8448        58
   macro avg     0.8498    0.8488    0.8423        58
weighted avg     0.8500    0.8448    0.8403        58

Confusion Matrix:
[[10  2  2  1]
 [ 0 13  1  0]
 [ 1  0 12  2]
 [ 0  0  0 14]]
✅ Saved model to saved_models\EEGNet_CSP_A08T.h5
✅ Saved metadata to saved_models\EEGNet_CSP_A08T_meta.json

=== Training on A09T ===
Extracting EDF parameters from c:\Users\hamih\OneDrive\Desktop\personal projects\BCI\BCICIV_2a_gdf\A09T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG

c:\Users\hamih\anaconda3\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)

Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
Event IDs found: {'1023': 1, '1072': 2, '276': 3, '277': 4, '32766': 5, '768': 6, '769': 7, '770': 8, '771': 9, '772': 10}
Using event_id for this subject: {'left_hand': 7, 'right_hand': 8, 'feet': 9, 'tongue': 10}
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Using data f


Classification Report:
              precision    recall  f1-score   support

           0     0.7500    1.0000    0.8571        15
           1     0.8333    0.7143    0.7692        14
           2     0.9000    0.6000    0.7200        15
           3     0.7500    0.8571    0.8000        14

    accuracy                         0.7931        58
   macro avg     0.8083    0.7929    0.7866        58
weighted avg     0.8089    0.7931    0.7867        58

Confusion Matrix:
[[15  0  0  0]
 [ 3 10  0  1]
 [ 2  1  9  3]
 [ 0  1  1 12]]
✅ Saved model to saved_models\EEGNet_CSP_A09T.h5
✅ Saved metadata to saved_models\EEGNet_CSP_A09T_meta.json
